In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the data
data = pd.read_csv('combined_data.csv', delimiter=';')

# Initialize LabelEncoder
le = LabelEncoder()

# Encode each categorical column
for column in data.columns[:-1]:  # Exclude the 'Label' column
    data[column] = le.fit_transform(data[column].astype(str))

# Encode the labels as well
data['Label'] = le.fit_transform(data['Label'].astype(str))

# Split the data into features (X) and target (y)
X = data.iloc[:, :-1].values  # All columns except the last one
y = data['Label'].values      # The last column is the target

# Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model using Keras
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy:.2f}')

# Save the model in TensorFlow SavedModel format
model.export('tensorflow_model')


Epoch 1/10


/Users/berkebozaci/.pyenv/versions/3.11.3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3333 - loss: 2.5923 - val_accuracy: 0.7620 - val_loss: 1.0370
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 788us/step - accuracy: 0.7997 - loss: 0.8598 - val_accuracy: 0.8240 - val_loss: 0.5326
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step - accuracy: 0.8640 - loss: 0.4659 - val_accuracy: 0.8780 - val_loss: 0.3823
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 670us/step - accuracy: 0.8884 - loss: 0.3500 - val_accuracy: 0.9000 - val_loss: 0.3034
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - accuracy: 0.9046 - loss: 0.2879 - val_accuracy: 0.9110 - val_loss: 0.2772
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step - accuracy: 0.9146 - loss: 0.2539 - val_accuracy: 0.9060 - val_loss: 0.2516
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - accuracy: 0.9306 - loss: 0.2118 - val_accuracy: 0.9050 - val_loss: 0.2476
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 688us/step - accuracy: 0.9299 - loss: 0.1997 - val_accuracy:

INFO:tensorflow:Assets written to: tensorflow_model/assets


Saved artifact at 'tensorflow_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10), dtype=tf.float32, name='keras_tensor_4')
Output Type:
  TensorSpec(shape=(None, 24), dtype=tf.float32, name=None)
Captures:
  14584563856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14584565200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14584565392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14584566160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14584565968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14584566928: TensorSpec(shape=(), dtype=tf.resource, name=None)


enter the command to convert `tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model --signature_name=serving_default --saved_model_tags=serve tensorflow_model/ model_tfjs/`
